<a href="https://colab.research.google.com/github/stabti/teaching_032025/blob/main/3-vanilla_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environnement d'exécution

Allez dans "Fichier" puis "enregistrer une copie dans Drive" pour sauvegarder vos modifications personnelles.

Attention, avant de commencer à exécuter le code, il faut choisir un environnement T4 si ce n'est déjà fait (cliquer sur la petite flèche à droite de RAM et Disque en haut à droite).

# Installation des packages
Ça peut prendre 2 bonnes minutes, c'est normal.

In [1]:
!pip install docling chonkie tiktoken sentence-transformers faiss-cpu transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.5/146.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 

# Import des packages

In [2]:
from docling.document_converter import DocumentConverter # For text extraction
from chonkie import TokenChunker # For chunking
from sentence_transformers import SentenceTransformer  # For embeddings
import faiss  # For vector database
import numpy as np  # For numerical operations
#from google.colab import files # to load a pdf file to test the system

# For answer generation with a small language model
# from transformers import pipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Chargement du PDF
Un bouton pour sélectionner le fichier à importer apparaît. Il suffit de faire la sélection.

In [4]:
#uploaded = files.upload()

# The uploaded file will be stored in a dictionary
#file_name = list(uploaded.keys())[0]  # Get the name of the uploaded file
file_name= "why_invest_AIEthics.pdf"

# Extraction du texte du PDF

In [5]:
def extract_text_from_pdf(file_name,output_file="output.md"):
  converter = DocumentConverter()
  result = converter.convert(file_name)
  text=result.document.export_to_markdown()
  with open(output_file, "w", encoding="utf-8") as f:
    f.write(text)
  print(f"Markdown content saved to: {output_file}")
  return text

Pour la cellule ci-dessous, pensez à bien mettre le bon nom de fichier PDF. L'extraction du texte prend un peu de temps (1mn environ), c'est normal (un modèle d'OCR est chargé pour analyser le document et cela est chronophage). Une fois terminé, on peut regarder le resultat en ouvrant le fichier markdown dans les documents (panneau de gauche).

In [7]:
### Extraction de text avec docling, attention
text = extract_text_from_pdf(file_name,"output.md")

/home/sonia/miniconda3/envs/rag/lib/python3.9/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Could not load the custom kernel for multi-scale deformable attention: /home/sonia/miniconda3/envs/rag/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /home/sonia/.cache/torch_extensions/py39_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so)
Could not load the custom kernel for multi-scale deformable attention: /home/sonia/miniconda3/envs/rag/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /home/sonia/.cache/torch_extensions/py39_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so)
Could not load the custom ke

Markdown content saved to: output.md


# Chunking
On découpe le texte extrait en petits morceaux (chunks) grâce au package Chonkie. Plusieurs techniques de chunking existent et il est bon d'en comparer plusieurs dans les cas d'usages en entreprise. Ici nous utilisons une méthode très simple qui découpe des chunks de taille chunk_size et avec un intersection de chunk_overlap entre chaque chunk.

In [8]:
def process_text(text):
  # Initialize the chunker
  chunker = TokenChunker(chunk_size=90, chunk_overlap=30) # defaults to using GPT2 tokenizer
  # Chunk the text
  chunks = chunker(text)
  return chunks


In [9]:
### tester le chunking
chunks= process_text(text)
# observer les chunks obtenus
for chunk in chunks:
    print(f"Chunk: {chunk.text}")
    print(f"Tokens: {chunk.token_count}")

Chunk: <!-- image -->

## Why invest in AI ethics and governance?

Five real-world origin stories

In collaboration with the Notre Dame-IBM Tech Ethics Lab

<!-- image -->

<!-- image -->

## How IBM can help

Clients can realize the potential of AI, analytics, and data using IBM's deep industry, functional, and technical expertise; enterprise-grade technology solutions; and
Tokens: 90
Chunk:  can realize the potential of AI, analytics, and data using IBM's deep industry, functional, and technical expertise; enterprise-grade technology solutions; and science-based research innovations.

For more information:

AI services from IBM Consulting ibm.com/services/artificial-intelligence AI solutions from IBM Software ibm.com/Watson AI innovations from IBM Research® research.ibm.com/artificial-intelligence

Tokens: 90
Chunk:  AI solutions from IBM Software ibm.com/Watson AI innovations from IBM Research® research.ibm.com/artificial-intelligence

The Notre Dame-IBM Tech Ethics Lab techethicsla

In [10]:
print(f"Total number of chunks: {len(chunks)}")

Total number of chunks: 140


# Retrieval system
On utilise un modèle d'embeddings issu de la librairie transformers de huggingface pour transformer chaque chunk en un vecteur numérique. On va indexer et stocker ces vecteurs grâce à la librairie FAISS développée par Meta qui facilitera également la recherche de vecteurs similaires dans la suite.

In [11]:
# Create retrieval system (FAISS index) with embeddings
def create_retriever(chunks):
    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    embeddings = model.encode(chunks, convert_to_tensor=True)
    embeddings_np = embeddings.cpu().numpy()

    dimension = embeddings_np.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings_np)
    return index, model

In [12]:
# Create retrieval system
index, model = create_retriever(chunks)

# Identification des chunks les plus pertinents par rapport à la query

In [13]:
# Retrieve relevant chunks based on query
def retrieve_info(index, model, query):
    # transformer la query en vecteur d'embedding
    query_embedding = model.encode(query, convert_to_tensor=True).cpu().numpy()
    # identifier les k chunks les plus pertinents pour cette query
    k = 3  # Number of nearest neighbors to retrieve
    scores, indices = index.search(query_embedding.reshape(1, -1), k)
    return scores, indices

# Example usage
# query = "votre demande en français ici"  # Replace with your French query
# scores, indices = retrieve_info(index, model, query)

In [20]:
# Query: Question à poser au système
query= "What are the three main takeaways of this document ?"

In [21]:
# Retrieve relevant chunks
scores, indices = retrieve_info(index, model, query)

In [22]:
# Regrouper les chunks les plus pertinents an un seul text qui constituera
# le context du prompt du modèle de language qui va générer une réponse à la question
retrieved_text = " ".join([chunks[idx].text for idx in indices[0]])
#answer = generate_answer(retrieved_text, query)
#print("\nAnswer:", answer)

In [23]:
print(retrieved_text)

abs/2309.13057. 'OECD AI Principles overview.' OECD. Accessed November 15, 2024. https://oecd.ai/en/ai-principles
- 12 The CEO's guide to generative AI: Customer and employee experience. IBM Institute for Business Value. August 2023. https://www.ibm.com/thoughtleadership/institute-business-value/en-us/report /ai-ethics-action

2

## Notes and

## sources

- 1 Goehring, Brian, Francesca Rossi, and Beth Rudden. AI ethics in action: An enterprise guide to progressing trustworthy AI . IBM Institute for Business Value. April 2022. https://ibm.co/ai-ethics-action
- 2 Ibid.
- 3 Ibid.
- 4 Ibid.
- /abs/2309.13057. OECD. (2023). OECD AI Principles overview. The OECD Artificial Intelligence (AI) Principles - OECD.AI. https:// oecd.ai/en/ai-principles
- 18 Dreksler, Noemi, David McCaffary, Lauren Kahn, Kate Mays, Markus Anderljung, Allan Dafoe, Michael C. Horowitz, and Baobao


# Génération d'une réponse à la question (query)
Grâce à un small language model, en utilisant les chunks les plus pertinents par rapport à la question (contexte) et en formulant un prompt qui concatène ce contexte et la question

In [24]:
# Chargement du modèle de language, ça peut prendre jusqu'à 1 minute environ

model_name = "ibm-granite/granite-3.1-2b-instruct"
# Chargement du tokenizer pour traiter le texte du prompt
tokenizer = AutoTokenizer.from_pretrained(model_name)
language_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="balanced",  # Using balanced CPU mapping.
    torch_dtype=torch.float16  # Use float16 if supported.
)
# mettre le modèle en mode "inférence"
language_model.eval()


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.18it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49155, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): GraniteRMSNorm((2048,)

In [25]:
# prompt = query + context
chat = [
        { "role": "user", "content": f"Answer the following question: \n\n{query}\n\n based on the following information: \n\n{retrieved_text}" },
    ]
chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to("cuda:0")
#input_tokens = tokenizer(chat, return_tensors="pt").to("cpu") # si le modèle fonctionne avec du cpu
# generate output tokens
output = language_model.generate(**input_tokens,max_new_tokens=512)
# decode output tokens into text
output = tokenizer.batch_decode(output)
# print output
print(output)

['<|start_of_role|>system<|end_of_role|>Knowledge Cutoff Date: April 2024.\nToday\'s Date: July 11, 2025.\nYou are Granite, developed by IBM. You are a helpful AI assistant.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>Answer the following question: \n\nWhat are the three main takeaways of this document?\n\n based on the following information: \n\nabs/2309.13057. \'OECD AI Principles overview.\' OECD. Accessed November 15, 2024. https://oecd.ai/en/ai-principles\n- 12 The CEO\'s guide to generative AI: Customer and employee experience. IBM Institute for Business Value. August 2023. https://www.ibm.com/thoughtleadership/institute-business-value/en-us/report /ai-ethics-action\n\n2\n\n## Notes and\n\n## sources\n\n- 1 Goehring, Brian, Francesca Rossi, and Beth Rudden. AI ethics in action: An enterprise guide to progressing trustworthy AI. IBM Institute for Business Value. April 2022. https://ibm.co/ai-ethics-action\n- 2 Ibid.\n- 3 Ibid.\n- 4 Ibid.\n- /abs/2309.13057. OECD. (2023). O

# Améliorations
Ce TP est un système de RAG ultra basique et a été crée à des fins pédagogiques. De nombreuses améliorations et expérimentations peuvent être menées en partant de cette base.

*   Rendre le code plus propre. Par exemple, faire un code qui utilise toutes les fonctions créées ci dessus et qui récapitule tout (d'ailleurs il faudrait créer une fonction pour la dernière cellule de chat). En effet, ici, après chaque fonction, j'ajoute un test pour des raisons pédagogiques mais normalement le code devrait être structuré différemment.
*   Tester d'autres techniques de chunking. Explorer la documentation de Chonkie et tester d'autres techniques, tester différents paramètres... Pourquoi pas utiliser autre chose que Chonkie aussi.
*   Améliorer le retrieval: énormément de choses peuvent être envisagées comme utiliser un autre modèle d'embedding. Pourquoi pas tester du reranking ?
*   Améliorer le prompt, en tester plusieurs, ajouter une étape de reformulation de prompt.
*   Tester d'autres modèles de language pour la génération de la réponse. Faire varier les paramètres.





